Machine learning potential in `OpenMM`
--------------------------------------

We have previously shown how to use OpenMM to run molecular dynamics (MD) simulations
with [molecular mechanics (MM)](https://en.wikipedia.org/wiki/Molecular_mechanics) force fields.

Despite remarkable speed ⚡️, MM oversimplifies the interactions among molecules as
low-order Fourier series, and harmonic and pairwise terms, and might not be accurate enough
on high energy region.

![MM](mm.png)

In this notebook, we replace the traditional MM in `OpenMM` with the popular
[machine learning potential](https://en.wikipedia.org/wiki/Machine_learning_potential) 
that have been rapidly developed in recent years.

First, we need to intall the `openmmml` package using `conda install -c conda-forge openmm-ml` to complement the `OpenMM` package with
ML potential functionality.

### First simulation with OpenMM-ML

In [35]:
import openmm
from openmm import app, unit
from openmmml import MLPotential

The [PDB](https://www.rcsb.org) loading stage is identical to the original `OpenMM` procedure.
Here we again load the good old alanine dipeptide example. 

In [36]:
import urllib
urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/openmm/openmm/master/wrappers/python/tests/systems/alanine-dipeptide-explicit.pdb",
    "input.pdb"
)
pdb = app.PDBFile('input.pdb')

In [37]:
from openmm.app import Modeller
modeller = Modeller(pdb.topology, pdb.positions)
modeller.deleteWater()

In lieu of a (MM) foce field, we here specify a machine learning potential.

In [38]:
potential = MLPotential('ani2x')

The creation of the system is again very similar to MM.
Note that here we don't need to specify specific solvent model etc. since the entire system
is treated without distinction by the ML potential.
In other words, the machine learning potential only sees a point cloud, without
paying attention to which part is solvent, protein, or ligand.

In [39]:
system = potential.createSystem(modeller.topology)

/Users/yuanqingwang/miniconda3/envs/openmm/lib/python3.11/site-packages/torchani/resources/
failed to equip `nnpops` with error: No module named 'NNPOps'


Now, to construct an integrator and a simulation is just like before.

In [40]:
#NOTE:HIDDEN
integrator = openmm.LangevinIntegrator(
    300*unit.kelvin, 
    1.0/unit.picoseconds, 
    2.0*unit.femtoseconds,
)

In [41]:
#NOTE:HIDDEN
simulation = app.Simulation(modeller.topology, system, integrator)

And just like before, you can easily set the positions
and minimize the energy.

In [43]:
#NOTE:HIDDEN
simulation.context.setPositions(modeller.positions)

In [44]:
simulation.minimizeEnergy()

Again, we employ a _Reporter_ object to record the simulation.

In [32]:
simulation.reporters.append(app.PDBReporter('output.pdb', 10))

We can now run the simulation.

In [45]:
simulation.step(100)

You can see that the time required to run this simulation (even without explicit water) is significantly longer than with traditional MM force fields.

In [48]:
urllib.request.urlretrieve(
    "http://www.quantum-machine.org/gdml/data/npz/ethanol_ccsd_t.zip",
    "ethanol_ccsd_t.zip"
)
import shutil
shutil.unpack_archive("ethanol_ccsd_t.zip", "ethanol_ccsd_t")

('ethanol_ccsd_t.zip', <http.client.HTTPMessage at 0x1685bf890>)

In [52]:
import numpy as np
data = np.load('ethanol_ccsd_t/ethanol_ccsd_t-test.npz')

In [57]:
list(data.keys())

['E', 'name', 'F', 'theory', 'R', 'z', 'type', 'md5']

In [58]:
data["z"]

array([6, 6, 8, 1, 1, 1, 1, 1, 1], dtype=uint8)

### Exercises
- Try a few systems of various sizes and observe how the time requirement scales with the size of the system.
And do the same for MM force fields.
- Try running this simulation with explicit water.